# Cross-resonance gate between transmon and fluxonium: linear approximation

In [7]:
import numpy as np
import time
import matplotlib.pyplot as plt
import pysqkit
from pysqkit import Qubit
from pysqkit.util.linalg import get_mat_elem
from functools import partial
import multiprocessing
from typing import List
import matplotlib
matplotlib.rcParams['mathtext.fontset'] = 'cm'
import util_tf_cr
import json
import cmath

In [8]:
with open('../../flx_transm_params.txt') as param_file:
    parameters_set = json.load(param_file)

In [9]:
p_set = "CR_3"

eps = 0.3

#Fluxonium
levels_f = 4

flx = pysqkit.qubits.Fluxonium(
    label='F', 
    charge_energy=parameters_set[p_set]["charge_energy_f"], 
    induct_energy=parameters_set[p_set]["induct_energy_f"], 
    joseph_energy=parameters_set[p_set]["joseph_energy_f"]
)
flx.diagonalize_basis(levels_f)
eigen_flx, eigstates_flx = flx.eig_states()

jc = parameters_set[p_set]["jc"]

# Generating data

In [11]:
def get_cr_effect(
    freq: float,
    eps: float,
    jc: float,
    flx: Qubit,
    eigen_flx: List[float],
    eigstates_flx: List[np.ndarray]
):
    """
    Description
    -----------------------------------------------------------------------
    Returns the exact cross-resonance coefficient in MHz.
    """
    
    
    
    transm_t = pysqkit.qubits.SimpleTransmon(
            label='T_tmp', 
            max_freq=freq, 
            anharm=-0.3,
            dim_hilbert=3,
        )
    
    f_03 = eigen_flx[3] - eigen_flx[0]
    q_03 = get_mat_elem(flx.charge_op(), eigstates_flx[0], eigstates_flx[3])
    f_12 = eigen_flx[2] - eigen_flx[1]
    q_12 = get_mat_elem(flx.charge_op(), eigstates_flx[1], eigstates_flx[2])
    freq_drive = transm_t.freq
    min_delta = np.min([np.abs(freq_drive - f_03)/q_03, np.abs(freq_drive - f_12)/q_12])
    eps_ratio = eps*np.max([np.abs(q_03/(freq_drive - f_03)), np.abs(q_12/(freq_drive - f_12))])
    
    
    coupled_sys = transm_t.couple_to(flx, coupling=pysqkit.couplers.capacitive_coupling, strength=jc)
    
    state_label = ["00", "01", "10", "11"]
    comp_states = {}
    for label in state_label:
        state_tmp = coupled_sys.state(label)[1]
        loc = np.argmax(np.abs(state_tmp))
        phase = cmath.phase(state_tmp[loc])
        state_tmp = np.exp(-1j*phase)*state_tmp
        comp_states[label] = state_tmp
    
    mu_cr = util_tf_cr.mu_yz_flx(comp_states, coupled_sys['F'].charge_op(), eps*1e3)
    
    mu_cr_sw = util_tf_cr.mu_yz_flx_sw(transm_t, flx, jc, eps*1e3)
    
    zz_coeff = util_tf_cr.zz(coupled_sys)*1e3
    
    output = {}
    output["freq_fixed"] = transm_t.freq
    output["freq"] = freq
    output["eps"] = eps
    output["eps_ratio"] = eps_ratio
    output["cr"] = np.abs(mu_cr)
    output["cr_sw"] = np.abs(mu_cr_sw)
    output["zz"] = zz_coeff
    output["gate_time"] = util_tf_cr.cr_gate_time(np.abs(mu_cr/1e3))
    output["gate_time_sw"] = util_tf_cr.cr_gate_time(np.abs(mu_cr_sw/1e3))
    
    output["jc"] = jc*1e3
    
    return output

func = partial(get_cr_effect, eps=eps, jc=jc, flx=flx, eigen_flx=eigen_flx, eigstates_flx=eigstates_flx)

In [12]:
start = time.time()

output_test = func(5.0)

end = time.time()

print("Computation time = {} s".format(end - start))



Computation time = 0.21795248985290527 s


In [13]:
output_test

{'freq_fixed': 5.0,
 'freq': 5.0,
 'eps': 0.3,
 'eps_ratio': 0.10779439345553458,
 'cr': 0.41981343032359647,
 'cr_sw': 0.45885498574159145,
 'zz': 0.11647912536272287,
 'gate_time': 297.7513127763653,
 'gate_time_sw': 272.4172208742109,
 'jc': 20.0}

In [14]:
delta_freq = 0.8
n_points = 200
ref_freq = 5.0
freq_list = list(np.linspace(ref_freq - delta_freq, ref_freq + delta_freq, n_points))

n_process = 1 # there seems to be issues with more than 1 n_process

start = time.time()

pool = multiprocessing.Pool(processes=n_process)

result = pool.map(func, freq_list)

pool.close()
pool.join()

end = time.time()

print("Computation time = {} s".format(end - start))

Computation time = 50.60101771354675 s


In [15]:
save = True
if save:
    with open('data_cr_transmon_fluxonium_linear.txt', 'w') as fp:
        json.dump(result, fp)